# M031(first half) and M025

In [1]:
import pandas as pd
import numpy as np

In [2]:
sub1 = pd.read_csv('../submissions/M025_0629_1311_submission_lgb_2folds_-1.3265CV_500000iter_0.1lr.csv')
sub2 = pd.read_csv('../temp/temp4of8_M031_0702_0822_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub3 = sub2.copy()

In [3]:
sub3.loc[sub2['scalar_coupling_constant'] == 0, 'scalar_coupling_constant'] = sub1.loc[sub2['scalar_coupling_constant'] == 0]

In [4]:
np.corrcoef(sub3['scalar_coupling_constant'], sub2['scalar_coupling_constant'])

array([[1.        , 0.99455264],
       [0.99455264, 1.        ]])

In [5]:
np.corrcoef(sub3['scalar_coupling_constant'], sub1['scalar_coupling_constant'])

array([[1.        , 0.99998192],
       [0.99998192, 1.        ]])

In [6]:
(sub3['scalar_coupling_constant'] == 0).sum()

0

In [7]:
oof1 = pd.read_csv('../oof/M025_0629_1311_oof_lgb_2folds_-1.3265CV_500000iter_0.1lr.csv')
oof2 = pd.read_csv('../temp/temp4of8_M031_0702_0822_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof3 = oof2.copy()

In [8]:
oof3.loc[oof2['oof_preds'] == 0, 'oof_preds'] = oof1.loc[oof2['oof_preds'] == 0]

In [9]:
oof3.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,88.266133
1,1,2JHH,-11.2570,-10.703280
2,2,2JHH,-11.2548,-10.703280
3,3,2JHH,-11.2543,-10.703280
4,4,1JHC,84.8074,88.266133


## OOF Scores

In [10]:
from sklearn.metrics import mean_absolute_error

for i, d in oof3.groupby('type'):
    print(i)
    print(mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds']))

1JHC
0.5792659784491359
1JHN
0.5595424991378696
2JHC
0.30022325062851835
2JHH
0.15434779537534812
2JHN
0.13000911360249118
3JHC
0.39051763625418157
3JHH
0.1934145785886842
3JHN
0.1627272677520273


In [11]:
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [12]:
group_mean_log_mae(oof3['scalar_coupling_constant'], oof3['oof_preds'], oof3['type'])

-1.3296796157097381

In [13]:
sub3.to_csv('../submissions/MERGE003-1J2J-M025-3J-M031_-1.3296CV.csv', index=False)